# ExpAn: Experiment Analysis

ExpAn is a Python library for the statistical analysis of randomised controlled trials (A/B tests). 

The functions are standalone and can be imported and used from within other projects, and from the command line.

The library is Open Source, published under the MIT license here:

[github.com/zalando/expan](https://github.com/zalando/expan)

# Installation

To install the library:

    $ pip install expan

For more information, start with the [README.rst](https://github.com/zalando/expan/blob/master/README.rst)

# ExpAn Architecture


## `data.loaders` seperate details of data from library

Loaders simply read the raw data (e.g. **`csv_fetcher.py`**) and return an `ExperimentData` object...

## `core.experimentdata` is the interface between loaders and library

The **`ExperimentData`** class abstracts the data structure such that we can have multiple loader modules that only need return this class, and none of our analysis modules need know anything about the file types.

## `core.experiment` provides the analysis functionality

Users of ExpAn will spend most of their time with the **`Experiment`** class, which extends `ExperimentData` with analysis functions that all return a `Results` object.

## `core.results` standardises all results

All analyses on an **`Experiment`** (`feature_check`, `deltaKPI`, `SGA`, `trend`, etc.) return a **`Results`** object, standardising the structure, and allowing a series of such analyses to be kept in one place.

## `core.statistics` contains underlying statistical functions

Such as: **`delta`**, **`bootstrap`**, **`chi-square`**. Used by higher-level `experiment` and `results` modules, and can be used directly from CLI, by passing in `Array`s.

## `core.binning` keeps binning separate from data

Instances of the **`Binning`** class (`NumericalBinning` and `CategoricalBinning`) represent a particular binning of some data, not the data itself, such that the same binning can then be applied to unseen data.

## `core.utils` is a just a grabbag of stuff

Pretty empty currently.

# Details of Components

Now we'll go into more details of the components, along with some examples of usage.

## `data.loaders`

Data loaders can be written as needed to handle different formats (CSV, Parquet, HDF5, etc) and different internal structures, so long as they return an `ExperimentData` object.

Currently, only a simply CSV loader (`data.csv_fetcher`) has been implemented.

We'll bypass this and work with synthesized data for now:

# Loading Data

In [1]:
import sys,os
from os.path import dirname, join, realpath
sys.path.insert(0, join(os.getcwd(), 'tests'))

import numpy as np
from tests_core.test_data import generate_random_data

np.random.seed(0)
data,metadata = generate_random_data()

ExpAn core init: v0.2.3


In [2]:
data.head()

,entity,variant,normal_same,normal_shifted,feature,normal_shifted_by_feature,treatment_start_time,normal_unequal_variance
0,0,A,-1.487862,-0.616148,non,-1.088533,8,0.413510
1,1,B,-1.125186,1.783682,has,1.167307,4,-3.212123
2,2,B,0.388819,1.007539,non,-1.055948,5,-5.986635
3,3,A,-1.173873,-0.889252,non,-0.152459,1,0.469914
4,4,A,1.112634,0.434377,has,0.175988,8,0.719719


In [3]:
metadata

{'experiment': 'random_data_generation',
 'primary_KPI': 'normal_shifted',
 'source': 'simulated'}

## core.experimentdata.ExperimentData

The `ExperimentData` class abstracts the data format so that none of our analysis modules need know anything about the underlying data structure.

In other words, this is the interface between fetching and analysis.

### metadata

Comprises *at least*:

* **`experiment`** - the name of the experiment (human-readable)
* **`source`** - a string indicated the source(s)
* **`primary_KPI`** - the Overall Evaluation Criteria *(not sure if this should be mandatory)*

and optionally:

* **`experiment_id`** - a machine-readable id for the experiment, if needed for your purposes
* **`retrieval_time`** - timestamp of when the data is fetched

It is here, in `ExperimentData`, that the fundamental distinction is made between KPIs and features.

They have different storage requirements, and different possible applications in the analysis.

Both are indexed by `entity`.

**Features** are all data that does not change throughout the experiment.

** KPIs** are all data that may be affected by the experiment. KPIs, therefore, may be indexed by `entity` and `date`.

# Constructing `ExperimentData`

Now we want to put the data we loaded manually into an `ExperimentData` object.

In [4]:
import expan

expdata = expan.experimentdata.ExperimentData(data, metadata)

In [5]:
print expdata

ExperimentData 'random_data_generation' with 2 features and 4 KPIs (primary: 'normal_shifted'), 10000 entities


The constructor automatically detects some features by name. All others are assumed to be KPIs...

*TODO: check that this is still true?*

In [14]:
print 'features: ' + ', '.join(expdata.feature_names)
print 'kpis    : ' + ', '.join(expdata.kpi_names)

features: treatment_start_time, feature
kpis    : normal_shifted_by_feature, normal_unequal_variance, normal_shifted, normal_same


## Features and KPIs are maintained as separate DataFrames

This is for two main reasons:
1. **Efficiency**: features never change over time, so need not be indexed as the KPIs will be
2. **Usage**: KPIs must never be used for a Sub-group Analysis (primarily because of Simpson's paradox)

However, the ExperimentData exposes both together (joined by `entity`) as the dynamic property `metrics`:

In [15]:
expdata.metrics.head()

,,normal_shifted,normal_unequal_variance,normal_same,normal_shifted_by_feature,treatment_start_time,feature
entity,variant,,,,,,
0,A,-0.616148,0.413510,-1.487862,-1.088533,8,non
1,B,1.783682,-3.212123,-1.125186,1.167307,4,has
2,B,1.007539,-5.986635,0.388819,-1.055948,5,non
3,A,-0.889252,0.469914,-1.173873,-0.152459,1,non
4,A,0.434377,0.719719,1.112634,0.175988,8,has


The constructor applies varies sanity checks:

In [16]:
expan.experimentdata.ExperimentData(data)

KeyError: 'ExperimentData requires metadata: [source,experiment]'

In [17]:
born_to_fail = data.drop('normal_shifted', axis=1)

expan.experimentdata.ExperimentData(born_to_fail, metadata)

KeyError: "ExperimentData requires the primary_KPI ('normal_shifted') to be present in the KPIs."

## core.experiment.Experiment (extends ExperimentData)

This class represents the experiment to be analysed.

It extends the `ExperimentData` class, adding analysis functionality.

## Constructing `Experiment` 

The `Experiment` class has one more requirement for metadata:

* **`baseline_variant`** - indicating which of the variants is to be considered baseline (a.k.a. control)

In our data we have two variants:

In [18]:
expdata.variant_names

{'A', 'B'}

So, with 'A' as the baseline variant, we construct an `Experiment` object.

In [19]:
exp = expan.experiment.Experiment('A', data, metadata)

print exp

Experiment 'random_data_generation' with 2 features and 4 KPIs (primary: 'normal_shifted'), 10000 entities
 2 variants: *A*, B


# Now we can start analysing!

## Let's start with a single DeltaKPI of orders:

In [21]:
import warnings
warnings.simplefilter('once',UserWarning)

In [22]:
res_delta = exp.delta(['normal_unequal_variance'])
print res_delta.metadata
res_delta.df

{'baseline_variant': 'A', 'version': '0.2.3', 'warnings': {'Experiment.delta': UserWarning('Sample variances differ too much to assume that population variances are equal.',)}, 'errors': {}, 'source': 'simulated', 'experiment': 'random_data_generation', 'primary_KPI': 'normal_shifted'}


value  \
variant                                                                          A   
metric                  subgroup_metric subgroup statistic     pctile                
normal_unequal_variance -               NaN      sample_size   NaN     6108.000000   
                                                 uplift        NaN        0.000000   
                                                 uplift_pctile 2.5       -0.035216   
                                                               97.5       0.035216   
                                                 variant_mean  NaN        0.005567   

                                                                                    
variant                                                                          B  
metric                  subgroup_metric subgroup statistic     pctile               
normal_unequal_variance -               NaN      sample_size   NaN     3892.000000  
                                                 uplift        NaN        0.015708  
                                                 uplift_pctile 2.5       -0.242713  
                                                               97.5       0.274130  
                                                 variant_mean  NaN        0.021275

Several things to see here.

### First, the warning

The analysis functions will emit warnings for known gotchas and strange things in the data.

*NB: currently, warnings aren't emitted but stored directly in Results. Will be changed: https://github.com/zalando/expan/issues/7*

### Second, the overkill output

This is the standardised `Result` structure. It seems redundant for a single DeltaKPI, but we will see later on why it is so convenient.

### What are our actual values:
The output to note here is:

* **`variant_mean` for both variants**: the individual means of each variant, for each metric
 * here, `0.005567` and `0.021275`

* **`uplift` for variant `B`**: this is simply the difference (delta) between the `variant_mean`s of `A` and `B`
 * here: an uplift of ~`0.016` in the KPI `normal_unequal_variance`

* **`uplift_pctile` for variant `B`**: this portrays the likely distribution of the uplift
 * In this case, the percentiles give the 95% confidence intervals (the 97.5% and 2.5% percentiles), indicating that:
 * the uplift may vary between `-0.24` and `+0.27`
 * we encode this distribution as two percentiles because it gives us flexibility in the future for various one-sided, two-sided, and partial-distribution outputs, also allowing Bayesian analyses on these outputs

### Metadata:
Also note that metadata from the `Experiment` is preserved in the `Result`.

## Using Bootstrapping:

The warning we got gives us concern that our normal assumption is violated. This suggests using a non-parametric approach, like bootstrapping.

We simply tell the `delta` function not to assume normality:

In [23]:
exp.delta(['normal_shifted'], assume_normal=False).df

value  \
variant                                                                 A   
metric         subgroup_metric subgroup statistic     pctile                
normal_shifted -               NaN      sample_size   NaN     6108.000000   
                                        uplift        NaN        0.000000   
                                        uplift_pctile 2.5       -0.034607   
                                                      97.5       0.033908   
                                        variant_mean  NaN       -0.005515   

                                                                           
variant                                                                 B  
metric         subgroup_metric subgroup statistic     pctile               
normal_shifted -               NaN      sample_size   NaN     3892.000000  
                                        uplift        NaN        0.990986  
                                        uplift_pctile 2.5        0.950993  
                                                      97.5       1.031373  
                                        variant_mean  NaN        0.985470

We don't notice it here, but bootstrapping takes considerably longer, so if we do not have an explicit reason to use it, it is almost always better to leave it off.

## Multiple metrics at once:

We don't actually have to analyse the metrics one at a time: the `delta` function (and most others) will by default operate on all KPIs:

In [24]:
import pandas as pd
pd.set_option('display.width', 80)
pd.set_option('display.precision', 7)

In [25]:
results = exp.delta()

In [26]:
results.df.reset_index(['subgroup','subgroup_metric'],drop=True)

value              
variant                                                    A             B
metric                    statistic     pctile                            
normal_unequal_variance   sample_size   NaN     6108.0000000  3892.0000000
                          uplift        NaN        0.0000000     0.0157081
                          uplift_pctile 2.5       -0.0352156    -0.2427134
                                        97.5       0.0352156     0.2741295
                          variant_mean  NaN        0.0055671     0.0212752
normal_shifted            sample_size   NaN     6108.0000000  3892.0000000
                          uplift        NaN        0.0000000     0.9909859
                          uplift_pctile 2.5       -0.0352283     0.9509697
                                        97.5       0.0352283     1.0310020
                          variant_mean  NaN       -0.0055154     0.9854704
normal_same               sample_size   NaN     6108.0000000  3892.0000000
                          uplift        NaN        0.0000000    -0.0330525
                          uplift_pctile 2.5       -0.0355484    -0.0732398
                                        97.5       0.0355484     0.0071347
                          variant_mean  NaN        0.0252194    -0.0078331
normal_shifted_by_feature sample_size   NaN     6108.0000000  3892.0000000
                          uplift        NaN        0.0000000     0.4622819
                          uplift_pctile 2.5       -0.0351568     0.4204035
                                        97.5       0.0351568     0.5041603
                          variant_mean  NaN        0.0164650     0.4787469

## core.results.Result

The `Result` class standardises the results of an analysis such as `feature_check`, `deltaKPI`, `SGA`, `trend`, or a series of such analyses.

Currently, this class **has-a** pandas `DataFrame`, but could in the future be implemented so that it **is-a** `DataFrame`, with the addition of metadata.

Above and beyond the structure it enforces, this class basically consists of:

1. Helper functions to ease indexing

1. Functions to restructure output of underlying functions into a `Result` 

In [28]:
#TODO: need to fix this https://github.com/zalando/expan/issues/9
print results

AttributeError: 'Results' object has no attribute 'means'

In [ ]:
#results.means()

In [ ]:
results.statistic('delta', 'sample_size', 'normal_shifted')

In [ ]:
#results.bounds()

In [29]:
import matplotlib.pyplot as pp
import seaborn
%matplotlib inline  

## Basing everything on pandas `DataFrame` gives us a lot for free:

In [ ]:
#results.bounds().loc[['normal_same','normal_shifted'],:].plot(kind='bar')

### Sub-Group Analysis (SGA)

Let's look at orders, net sales, and PCII but broken down by CLV subgroups...

In [ ]:
sga_results = exp.sga(['feature'],['normal_shifted'])

In [ ]:
pd.set_option('display.max_rows',200)
#Note: this if using HTML output, truncation causes the index to be screwed up
sga_results.df

### Trend Analysis 

In [ ]:
#Will show time-domain analysis later when I load time data
# Create time column. TODO: Do this nicer
exp.kpis['time_since_treatment'] = exp.features['treatment_start_time']
# Make time part of index
exp.kpis.set_index('time_since_treatment',append=True,inplace=True)

In [ ]:
pd.set_option('display.width', 120)
res_trend = exp.trend(['normal_shifted'])
res_trend.df

## core.binning

More complicated than you'd think.

Defines a Binning class that represents a particular binning of a data, such that the same binning can then be applied to unseen data.

Numerical and Categorical Binnings are defined.

Tries to handle skewed data.

In [ ]:
a = exp.features.xs(('A'),level=('variant'))
b = exp.features.xs(('B'),level=('variant'))

### Now we create the binning

This simply determines the thresholds appropriate for creating the requested number of bins...

In [ ]:
import core.binning as binning
binning.dbg_lvl=0

bins = binning.create_binning(a.loc[:,'treatment_start_time'])

print bins

### We can *apply* this binning to the same data:

In [ ]:
a_bins = bins.label(a.treatment_start_time)

pd.DataFrame(a_bins).head(10)

### And we can *apply* it to different data:

In [ ]:
b_bins = bins.label(b.treatment_start_time)

pd.DataFrame(b_bins).head(10)

Note that there is a hidden 'catch-all' bin...

This is implemented as the last entry in the arrays, making indexing very easy: an unknown bin is always -1.

In [ ]:
bins.uppers

In [ ]:
bins._uppers

### Bin labels can be arbitrarily formatted:

Without running the binning algorithm on the data again.

In [ ]:
print bins

In [ ]:
print bins.__str__('{conditions}')

In [ ]:
print bins.__str__('{iter.uppercase}')

In [ ]:
print bins.__str__('{iter.uppercase}: From {lo:.2f} \t To {up:.2f}')

## core.statistics

Here the underlying statistical functions are implemented. These are used by the higher-level `experiment` and `results` modules, and can indeed be used directly by passing in NumPy `Array`s.

The more interesting functions are:

### `bootstrap`

Bootstraps the Confidence Intervals for a particular function comparing two samples. NaNs are ignored (discarded before calculation).

This function, as well as others such as `normal_sample_difference`, and `delta`, take as input a list of percentiles, and return the values corresponding to those percentiles. This implementation is very general, allowing us to use the same functions for one-sided as well as two-sided tests, as well as more exactly recreating an output distribution (e.g. if we want to graphically depict more than 95% confidence intervals).

### `delta`

Uses either bootstrap or standard normal assumptions to compute the difference between two arrays.

## core.utils

A `commons` module: a grabbag of stuff we use all over the place.

Pretty empty currently.

# That's it! Try it out for yourself:


[github.com/zalando/expan](https://github.com/zalando/expan)